In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

import sys
sys.path.append('../src')
import utils_explore as ue

data = ue.all_bets()

In [ ]:
#     'ostapbender', 'nvaluev', 'karpovvyacheslav', 'naguchevroman'
##    'chaplygin', 'kozhevnikov', 'netsenko', 'zhukov', 'falcao1984', 'andronov', 'teplofevralya', 'nobel'

In [ ]:
#filter group
df_tmp = data[data.author.isin(['netsenko', 'kozhevnikov', 'chaplygin'])]
df_tmp = ue.calc_win_rate(df_tmp, period_split='M') #[['author','date', 'count', 'success_rate']]

In [ ]:
df_plot = df_tmp.pivot_table(index=['placed-date'], columns='author', values=['count', 'success_rate'])
df_plot.plot()
df_plot.describe()

In [ ]:
df_plot_19 = df_tmp[df_tmp['placed-date'] >= datetime.date(2019, 1, 1)]
df_plot_19.plot()
df_plot_19.describe()